In [2]:
import sys
sys.path.append('../personal_pkgs/')

import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

#### 저널 별 데이터를 모으자

In [3]:
fm_txt_ls = pars.journal_txt_ls('한국재무관리학회')
sc_txt_ls = pars.journal_txt_ls('한국증권학회지')
dr_txt_ls = pars.journal_txt_ls('한국파생상품학회')
f_txt_ls = pars.journal_txt_ls('한국재무학회')

In [4]:
fm_path = [refer for i in fm_txt_ls for refer in i]
f_path = [refer for i in f_txt_ls for refer in i]
sc_path = [refer for i in sc_txt_ls for refer in i]
dr_path = [refer for i in dr_txt_ls for refer in i]

fm_sent_ls , fm_error_ls = pars.split_to_sent(fm_path)
f_sent_ls , f_error_ls = pars.split_to_sent(f_path)
sc_sent_ls , sc_error_ls = pars.split_to_sent(sc_path)
dr_sent_ls , dr_error_ls = pars.split_to_sent(dr_path)

In [5]:
len(fm_sent_ls) , len(f_sent_ls) , len(sc_sent_ls) , len(dr_sent_ls)

(11487, 7213, 9578, 8166)

In [6]:
len(fm_error_ls) , len(f_error_ls) , len(sc_error_ls) , len(dr_error_ls)

(99, 42, 78, 76)

In [10]:
fm_error_path_ls = [fm_error_ls[idx][1] for idx in range(len(fm_error_ls))]
f_error_path_ls = [f_error_ls[idx][1] for idx in range(len(f_error_ls))]
sc_error_path_ls = [sc_error_ls[idx][1] for idx in range(len(sc_error_ls))]
dr_error_path_ls = [dr_error_ls[idx][1] for idx in range(len(dr_error_ls))]

In [19]:
fm_error_df = pd.DataFrame(fm_error_path_ls)
f_error_df = pd.DataFrame(f_error_path_ls)
sc_error_df = pd.DataFrame(dr_error_path_ls)
dr_error_df = pd.DataFrame(sc_error_path_ls)

pd.concat([fm_error_df,f_error_df,sc_error_df,dr_error_df]).to_csv('error_path.csv',index=False)

- `*_q_ls` : double quotation 이 있는 sentence. 분석 가능한 문장이라고 간주해 dictionary 를 형성하는 데 사용된다.
- `*_nq_ls` : quotation 이 없는 sentence. 불안정한 문장, 즉, 딕셔너리를 형성하는데에는 부적합하다고 간주한다.

In [6]:
fm_q_ls , fm_nq_ls = pars.show_quote_and_unquote(fm_sent_ls)
f_q_ls , f_nq_ls = pars.show_quote_and_unquote(f_sent_ls)
sc_q_ls , sc_nq_ls = pars.show_quote_and_unquote(sc_sent_ls)
dr_q_ls , dr_nq_ls = pars.show_quote_and_unquote(dr_sent_ls)

ratio of double quotation line : 0.8318969269609123
total length of double quote lines: 9556
total length of lines: 11487
ratio of double quotation line : 0.6974906418965756
total length of double quote lines: 5031
total length of lines: 7213
ratio of double quotation line : 0.3106076425140948
total length of double quote lines: 2975
total length of lines: 9578
ratio of double quotation line : 0.24222385500857213
total length of double quote lines: 1978
total length of lines: 8166


In [7]:
fm_ip_ls = pars.make_imperfect_ls(fm_nq_ls)
f_ip_ls = pars.make_imperfect_ls(f_nq_ls)
sc_ip_ls = pars.make_imperfect_ls(sc_nq_ls)
dr_ip_ls = pars.make_imperfect_ls(dr_nq_ls)

#### 딕셔너리 만들기

In [8]:
fm_df,unique_fm_df = pars.make_dictionary(fm_q_ls)
f_df,unique_f_df = pars.make_dictionary(f_q_ls)
sc_df,unique_sc_df = pars.make_dictionary(sc_q_ls)
dr_df,unique_dr_df = pars.make_dictionary(dr_q_ls)

eliminate the duplicated paper name
(9556, 4)
(8067, 5)
eliminate the duplicated paper name
(5031, 4)
(4317, 5)
eliminate the duplicated paper name
(2975, 4)
(2338, 5)
eliminate the duplicated paper name
(1978, 4)
(1735, 5)


In [9]:
fm_df['from'] = ['한국재무관리학회' for _ in range(len(fm_df))]
f_df['from'] = ['한국재무학회' for _ in range(len(f_df))]
sc_df['from'] = ['한국증권학회' for _ in range(len(sc_df))]
dr_df['from'] = ['한국파생상품학회' for _ in range(len(dr_df))]

In [10]:
dictionary_df = pd.concat([fm_df,f_df,sc_df,dr_df],ignore_index=True)
dictionary_df.shape

(19540, 5)

#### 점수 기반해서 quotation 이 없는 sentence 에 대해서 논문 이름을 추출한다

In [11]:
idx_ls = []
print(dictionary_df.shape)

for idx,val in enumerate(dictionary_df['paper'].tolist()) :
    if val not in [i[1] for i in idx_ls] :
        idx_ls.append((idx,val))

quote_df = dictionary_df.iloc[[i[0] for i in idx_ls]]
quote_df.reset_index(inplace=True)
print(quote_df.shape )

(19540, 5)
(14609, 6)


#### 0.9 라는 similarity score 를 기준으로 삼아서, double quoatation 이 없는 문장의 paper name 을 catch 해준다.

In [12]:
fm_catch_paper = pars.catch_the_paper_under_similarity_score(quote_df,fm_ip_ls,score=0.9)
f_catch_paper = pars.catch_the_paper_under_similarity_score(quote_df,f_ip_ls,score=0.9)
sc_catch_paper = pars.catch_the_paper_under_similarity_score(quote_df,sc_ip_ls,score=0.9)
dr_catch_paper = pars.catch_the_paper_under_similarity_score(quote_df,dr_ip_ls,score=0.9)

the length of dictionary is 14609.. the cost of function is proportional with it
0
4000
8000
12000
the length of dictionary is 14609.. the cost of function is proportional with it
0
4000
8000
12000
the length of dictionary is 14609.. the cost of function is proportional with it
0
4000
8000
12000
the length of dictionary is 14609.. the cost of function is proportional with it
0
4000
8000
12000


In [39]:
print(len(fm_catch_paper),
len(f_catch_paper),
len(sc_catch_paper),
len(dr_catch_paper))

164 109 2732 417


총합친 것도 필요하지만, 우리에게 필요한 것은 저널별 데이터이다.

In [40]:
fm_detect_ls = [i[0] for i in fm_catch_paper]
f_detect_ls = [i[0] for i in f_catch_paper]
sc_detect_ls = [i[0] for i in sc_catch_paper]
dr_detect_ls = [i[0] for i in dr_catch_paper]

In [41]:
fm_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국재무관리학회']['paper'].tolist()
fm_paper_name_ls = fm_paper_name_ls + fm_detect_ls
len(fm_paper_name_ls)

9720

In [42]:
f_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국재무학회']['paper'].tolist()
f_paper_name_ls = f_paper_name_ls + f_detect_ls
len(f_paper_name_ls)

5140

In [43]:
sc_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국증권학회']['paper'].tolist()
sc_paper_name_ls = sc_paper_name_ls + sc_detect_ls
len(sc_paper_name_ls)

5707

In [44]:
dr_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국파생상품학회']['paper'].tolist()
dr_paper_name_ls = dr_paper_name_ls + dr_detect_ls
len(dr_paper_name_ls)

2395

In [45]:
fm_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(fm_paper_name_ls)) : 
    if len(fm_final_df) % 4000 == 0 : print(len(fm_final_df))
    append_df = quote_df[quote_df['paper'] == fm_paper_name_ls[idx]]
    fm_final_df = pd.concat([fm_final_df,append_df])

0
4000
8000


In [20]:
fm_final_df.to_csv('fm_base_df.csv',index=False)

In [46]:
f_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(f_paper_name_ls)) : 
    if len(f_final_df) % 4000 == 0 : print(len(f_final_df))
    append_df = quote_df[quote_df['paper'] == f_paper_name_ls[idx]]
    f_final_df = pd.concat([f_final_df,append_df])

0
4000


In [47]:
f_final_df.to_csv('f_base_df.csv',index=False)

In [48]:
sc_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(sc_paper_name_ls)) : 
    if len(sc_final_df) % 4000 == 0 : print(len(sc_final_df))
    append_df = quote_df[quote_df['paper'] == sc_paper_name_ls[idx]]
    sc_final_df = pd.concat([sc_final_df,append_df])

0
4000


In [49]:
sc_final_df.to_csv('sc_base_df.csv',index=False)

In [50]:
dr_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(dr_paper_name_ls)) : 
    if len(dr_final_df) % 4000 == 0 : print(len(dr_final_df))
    append_df = quote_df[quote_df['paper'] == dr_paper_name_ls[idx]]
    dr_final_df = pd.concat([dr_final_df,append_df])

0


In [51]:
dr_final_df.to_csv('dr_base_df.csv',index=False)

#### catch_paper_name_ls 은 저널에 관계없이 non-quoted sentence 에서 총 어떤 논문 이름을 잡았는지에 대한 데이터를 담은 리스트이다.

In [52]:
catch_paper_name_ls = [i[0] for i in fm_catch_paper]+\
[i[0] for i in f_catch_paper]+\
[i[0] for i in sc_catch_paper]+\
[i[0] for i in dr_catch_paper]
len(catch_paper_name_ls)

3422

- final_paper_name_ls 는 paper name 으로 uniquely indexing 해주기 전의 double quotation sentence 를 담은 dataframe 이다.
- non double quotation sentence 데이터에서 추출한 catch_paper_name_ls를 dictionary_df 의 인덱스와 합치면 총 논문 이름에 대한 리스트를 얻게 된다.

In [53]:
base_paper_name_ls = dictionary_df['paper'].tolist()
final_paper_name_ls = base_paper_name_ls + catch_paper_name_ls
len(final_paper_name_ls)

22962

final_paper_name_ls 의 논문 이름 포맷은 전적으로 quote_df 를 따른다. 이에 따라서, Counting 을 할 때, quote_df 의 포맷을 유지하면서 `expanding` 시켜준다.

In [54]:
base_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(final_paper_name_ls)) : 
    if len(base_df) % 4000 == 0 : print(len(base_df))
    append_df = quote_df[quote_df['paper'] == final_paper_name_ls[idx]]
    base_df = pd.concat([base_df,append_df])

0
4000
8000
12000
16000
20000


In [55]:
base_df.to_csv('base_df.csv',index=False)

- `base_df` 는 저널에 관계없이 전체 논문에 대해서 similarity score를 기반으로 최대한 merge 한 데이터들을 담은 dataframe 이다.
- `{}_base_df` 는 {} 에 붙은 alias 에 따라 특정 논문에 대한 base_df 이다.